In [6]:
import cv2
from datetime import datetime
import pandas as pd
from bokeh.plotting import figure, show, output_file

In [7]:
video = cv2.VideoCapture(0)

first_frame = None

status_list = [None, None]
times = []
start=[]
end = []
while True:
    
    check, frame = video.read()
    status = 0
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (15,15), 0)
    if first_frame is None:
        first_frame = gray
        continue
    
    
    delta_frame = cv2.absdiff(first_frame, gray)
    thresh_frame = cv2.threshold(delta_frame, 130, 255, cv2.THRESH_BINARY)[1]
    thresh_frame = cv2.dilate(thresh_frame, None, iterations=2)
    
    
    [cnts,_] = cv2.findContours(thresh_frame.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for cnt in cnts:
        if cv2.contourArea(cnt)>10000:
            status = 1
            status_list.append(status)
            (x,y,w,h) = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 3)
    
    status_list.append(status)
    status_list = status_list[-2:]
    
    if status_list[-2]==0 and status_list[-1]==1:
        times.append(datetime.now())
    if status_list[-2]==1 and status_list[-1]==0:
        times.append(datetime.now())
    
    cv2.imshow("frame", frame)
    cv2.imshow("delta_frame", delta_frame)
    cv2.imshow("thresh_frame", thresh_frame)
    key = cv2.waitKey(1)
    if key==ord('q'):
        if status==1:
            times.append(datetime.now())
        break

video.release()
cv2.destroyAllWindows()

df = pd.DataFrame(columns=["Start", "End"])

for i in range(0,len(times)):
    if i%2 == 0:
        start.append(times[i])
    else:
        end.append(times[i])

for i in range(0,len(end)):
    df = df.append({"Start": start[i], "End": end[i]}, ignore_index=True)
df.to_csv("times.csv")

KeyboardInterrupt: 

In [ ]:
df = pd.read_csv("times.csv", parse_dates=["Start", "End"])
p = figure(width=500, height=100, x_axis_type="datetime", sizing_mode="scale_both")
q = p.quad(left=df["Start"], right=df["End"],bottom=0, top=1, color="green")
output_file("motion_graph.html")
show(p)